Imports

In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from collections import OrderedDict
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


Load Dataset

In [81]:
bmr_df = pd.read_csv('./BMR8833.csv')
meals_df = pd.read_csv('./Meals.csv')


Clean Data And Encoding 

In [82]:
bmr_df['Gender'] = bmr_df['Gender'].map({'Male': 0, 'Female': 1})

bmr_df.fillna('missing', inplace=True)
meals_df.fillna('No_Snack', inplace=True)


 Convert numerical columns to integers

In [83]:

numerical_cols = ['Age', 'Weight(kg)', 'Height(cm)', 'BMI', 'Basal_Metabolic_Rate(BMR)']
bmr_df[numerical_cols] = bmr_df[numerical_cols].astype(int)

if 'BMI_Index' in bmr_df.columns:
    bmr_df['BMI_Index'] = pd.Categorical(bmr_df['BMI_Index']).codes

Split Data For BMR Prediction

In [84]:

X_bmr = bmr_df.drop(columns=['Basal_Metabolic_Rate(BMR)'])
y_bmr = bmr_df['Basal_Metabolic_Rate(BMR)']
X_train_bmr, X_test_bmr, y_train_bmr, y_test_bmr = train_test_split(X_bmr, y_bmr, test_size=0.2, random_state=42)



Train a regression model to predict BMR


In [85]:
bmr_model = LinearRegression()
bmr_model.fit(X_train_bmr, y_train_bmr)

LinearRegression()


Evaluate BMR model


In [86]:
y_pred_bmr = bmr_model.predict(X_test_bmr)
mae = mean_absolute_error(y_test_bmr, y_pred_bmr)
mse = mean_squared_error(y_test_bmr, y_pred_bmr)
r2 = r2_score(y_test_bmr, y_pred_bmr)

Print Values of Evaluations

In [87]:

print("BMR Prediction Model Evaluation:")
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

BMR Prediction Model Evaluation:
Mean Absolute Error: 454.04360379833764
Mean Squared Error: 279522.93184025184
R-squared: 0.38264867735629526


 Save the model to a pickle file

In [88]:

filename = 'bmr_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(bmr_model, file)

 Load the model from the pickle file

In [89]:

with open(filename, 'rb') as file:
    bmr_model = pickle.load(file)


Split Meals Dataset Drop Cal

In [90]:

meals_df.drop(columns=["Calories"], inplace=True)

 Predict BMR using the trained model

In [91]:

meals_df['Predicted_BMR'] = bmr_model.predict(X_bmr)


Specify columns for which you want to get unique values

In [92]:
columns_to_check = ['Breakfast', 'Lunch', 'Dinner', 'Snack', 'Snack2']

unique_values_with_index = {}

for column in columns_to_check:
    unique_values_ordered = list(OrderedDict.fromkeys(meals_df[column]))
    value_index_dict = {value: index for index, value in enumerate(unique_values_ordered)}
    unique_values_with_index[column] = value_index_dict

for column, encoding_dict in unique_values_with_index.items():
    meals_df[column] = meals_df[column].map(encoding_dict)

Split Meal Dataset 

In [93]:
X_Meal = meals_df.drop(columns=['Breakfast', 'Lunch', 'Dinner', 'Snack', 'Snack2'])
y_Meal = meals_df[['Breakfast', 'Lunch', 'Dinner', 'Snack', 'Snack2']]
X_train_Meal, X_test_Meal, y_train_Meal, y_test_Meal = train_test_split(X_Meal, y_Meal, test_size=0.2, random_state=42)

In [94]:
X_train_Meal

,NumberOfMeals,Predicted_BMR
5873,5,2773.428013
7051,4,2999.341657
8452,4,2249.523532
2908,4,2428.455600
665,5,2941.384588
...,...,...
5734,4,2307.026669
5191,4,1995.467429
5390,5,1607.552223
860,5,2807.034256


In [95]:
y_train_Meal

,Breakfast,Lunch,Dinner,Snack,Snack2
5873,269,16,20,126,0
7051,88,15,138,204,143
8452,489,96,103,5,179
2908,148,133,13,15,0
665,27,74,101,23,14
...,...,...,...,...,...
5734,129,46,78,5,23
5191,137,86,104,59,99
5390,34,143,4,67,0
860,38,29,142,78,106


Applay ANN For Meal

# Define the model architecture for meal prediction

In [96]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    Dense(128, activation='relu'),
    Dense(5, activation='sigmoid')
])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

batch_size = 32
epochs = 6
model.fit(X_train_Meal, y_train_Meal, batch_size=batch_size, epochs=epochs, validation_split=0.2)



c:\Users\HodaE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,157 (35.77 KB)

 Trainable params: 9,157 (35.77 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
177/177 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4617 - loss: 326153.9062 - val_accuracy: 0.5449 - val_loss: 2268810.0000
Epoch 2/6
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5434 - loss: 3836124.5000 - val_accuracy: 0.5449 - val_loss: 10800558.0000
Epoch 3/6
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5460 - loss: 13988107.0000 - val_accuracy: 0.5449 - val_loss: 25883444.0000
Epoch 4/6
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5459 - loss: 30772108.0000 - val_accuracy: 0.5449 - val_loss: 51817272.0000
Epoch 5/6
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5314 - loss: 59313820.0000 - val_accuracy: 0.5449 - val_loss: 83412664.0000
Epoch 6/6
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5477 - loss: 95508424.0000 - val_accuracy: 0.5449 - val_loss: 126489376.0000


 Evaluate the model

In [97]:

loss, accuracy = model.evaluate(X_test_Meal, y_test_Meal)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5243 - loss: 126044592.0000
Test Loss: 126648488.0
Test Accuracy: 0.5212224125862122


Define the find_key_for_value function   To Return Orignal Data

In [98]:

def find_key_for_value(encoding_dicts, column_name, value):
    if column_name in encoding_dicts:
        encoding_dict = encoding_dicts[column_name]
        for key, val in encoding_dict.items():
            if val == value:
                return key
    return None 


Get user input for BMR features

In [99]:

age = int(input("Enter Age: "))
weight = int(input("Enter Weight (kg): "))
height = int(input("Enter Height (cm): "))
gender = input("Enter Gender (Male/Female): ")
bmi = float(input("Enter BMI: "))
bmi_index = input("Enter BMI Index (Underweight/Normal/Overweight/Obese): ")


Map gender and BMI index to numerical values

In [100]:
gender_mapping = {'Male': 0, 'Female': 1}
bmi_index_mapping = {'Underweight': 0, 'Normal': 1, 'Overweight': 2, 'Obese': 3}

gender_numeric = gender_mapping.get(gender, -1)
bmi_index_numeric = bmi_index_mapping.get(bmi_index, -1)

if gender_numeric == -1 or bmi_index_numeric == -1:
    print("Invalid input for Gender or BMI Index.")
else:
    # Predict BMR using the trained model
    bmr_prediction = bmr_model.predict([[age, weight, height, gender_numeric, bmi, bmi_index_numeric]])
    print(f"Predicted Basal Metabolic Rate (BMR): {bmr_prediction[0]}")



Predicted Basal Metabolic Rate (BMR): 1733.3682411932284


c:\Users\HodaE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


 Display recommended meals for the current number of meals

In [101]:

# for num_meals in [3, 4, 5]:
#     meal_prediction = model.predict([[bmr_prediction[0], num_meals]])

#     print(f"\nRecommended Meals for {num_meals} meals:")
#     recommended_meals = meals_df[meals_df['NumberOfMeals'] == num_meals].iloc[0]
#     meals_to_display = ['Breakfast', 'Lunch', 'Dinner']
#     if num_meals >= 4:
#         meals_to_display.append('Snack')
#     if num_meals == 5:
#         meals_to_display.append('Snack2')
#     for meal in meals_to_display:
#         meal_name = recommended_meals[meal]
#         if pd.notna(meal_name) and meal_name != 'missing':
#             original_category = find_key_for_value(unique_values_with_index, meal, meal_name)
#             print(f"{meal}: {original_category}")
#     print('===================================')


In [103]:


for num_meals in [3, 4, 5]:
    bmr_prediction_input = np.array([[bmr_prediction[0], num_meals]])
    meal_prediction = model.predict(bmr_prediction_input)

    print(f"\nRecommended Meals for {num_meals} meals:")
    recommended_meals = meals_df[meals_df['NumberOfMeals'] == num_meals].iloc[0]
    meals_to_display = ['Breakfast', 'Lunch', 'Dinner']
    if num_meals >= 4:
        meals_to_display.append('Snack')
    if num_meals == 5:
        meals_to_display.append('Snack2')
    for meal in meals_to_display:
        meal_name = recommended_meals[meal]
        if pd.notna(meal_name) and meal_name != 'missing':
            original_category = find_key_for_value(unique_values_with_index, meal, meal_name)
            print(f"{meal}: {original_category}")
    print('===================================')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

Recommended Meals for 3 meals:
Breakfast: Honey Avocado Smoothie
Lunch: Goat Cheese on Toasted Bread with Tomato
Dinner: Chicken Celery Sticks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step

Recommended Meals for 4 meals:
Breakfast: Spinach, Onion, Mushroom, and Bell Pepper Egg White Omelet
Lunch: Guacamole on Tostada
Dinner: Chocolate Almond Milk Protein Shake
Snack: High Potassium Fruit Salad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Recommended Meals for 5 meals:
Breakfast: Egg White and Mushroom Omelet
Lunch: Rice Cake with Strawberries and Honey
Dinner: Tuna Mex Tuna Salad
Snack: Mixed Greens with Sun-Dried Tomato Dressing
Snack2: Very Green Veggie Protein Smoothie
